<a href="https://colab.research.google.com/github/37stu37/rnc2/blob/master/temporal_network_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import sys
import os
from pathlib import Path
import networkx as nx
from numba import jit
from dask.distributed import Client
pd.options.mode.chained_assignment = None  # default='warn'

client = Client(processes=False)
client

/usr/local/lib/python3.6/dist-packages/distributed/bokeh/core.py:57: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 99] Cannot assign requested address
  warnings.warn('\n' + msg)
/usr/local/lib/python3.6/dist-packages/distributed/deploy/local.py:197: UserWarning: 
Could not launch service 'bokeh' on port 8787. Got the following message:

[Errno 99] Cannot assign requested address
  self.scheduler.start(scheduler_address)


Client Scheduler: inproc://172.28.0.2/125/13,Cluster Workers: 1 Cores: 2 Memory: 13.65 GB




---


**Tentative code for rnc2 temporal network**

@time

Conditions -> Boolean mask -> Contacts -> Edgelist


---



In [0]:
# import data
folder = Path('/content/drive/My Drive/04_Cloud/01_Work/Academia/01_Publications/00_Alex/005_RNC2')
edge_file = folder / 'data' / 'Copy of edge_data.parquet'
wind_file = folder / 'data' / 'Copy of GD_wind.csv'

wind_data = pd.read_csv(wind_file) 
edgelist = pd.read_parquet(edge_file, engine='pyarrow')

**Conditions**


---



In [0]:
def wind_scenario(wind_data):  
    i = np.random.randint(0, wind_data.values.shape[0])
    w = wind_data.values[i, 2]
    dist = wind_data.values[i, 1]
    b = wind_data.values[i, 3]
    bear_max = b + 45  # wind direction
    bear_min = b - 45
    if b == 360:
        bear_max = 45
    if b <= 0:  # should not be necessary
        bear_min = 0
    if b == 999:
        bear_max = 999
        bear_min = 0
    return bear_max, bear_min, dist # wind characteristics, bearing and distance

# ignition ?
def mask_ignition_rng(e):
    rng = np.random.uniform(0, 1, size=e.values.shape[0])
    mask = rng < e.IgnProb_bl.values
    return mask*1

# wind direction
def mask_wind_direction(e, w_bear_max, w_bear_min):
    mask = (e.bearing.values < w_bear_max) & (e.bearing.values > w_bear_min) 
    return mask*1

# wind speed
def mask_wind_speed(e, w_dist):
    mask = e.distance.values < w_dist
    return mask*1

# not burnt in previous fires ?
def mask_burnt(t, e, c):
    if t != 1:
      previouslyActivated = c[:,:t-1].sum(axis=1)
    else:
      previouslyActivated = c[:,0]
    mask = np.where(previouslyActivated > 0, 1, 0)
    # mask = mask == 0 # == 0 means that no fire previously occured
    return mask*1

# target -> source ?
def mask_target_to_source(t, e, c):
    previousTargetAtTime = e.target[c[:, t-1] == 1]
    mask = np.in1d(e.source.values, previousTargetAtTime.values)
    return mask*1

**Main**


---



In [215]:
for scenario in range(2):
    time = 0 
    # initial conditions
    m_ignition_rng = mask_ignition_rng(edgelist)
    print("{} ignitions : {} over {} total ~ {} % of buildings".format(m_ignition_rng.sum(axis=0), m_ignition_rng[:5], edgelist.shape[0], 
                                                        (m_ignition_rng.sum(axis=0) / edgelist.shape[0])* 100))
    ActTime = np.zeros((m_ignition_rng.shape[0],1))
    ActTime[:, 0] = m_ignition_rng
    # wind conditions and mask
    w_direction_max, w_direction_min, w_distance = wind_scenario(wind_data)
    m_wind_direction = mask_wind_direction(edgelist, w_direction_max, w_direction_min)
    m_wind_speed = mask_wind_speed(edgelist, w_distance)
    m_wind = m_wind_direction & m_wind_speed
    # print("wind_direction : {}, wind_speed : {}, wind : {}".format(m_wind_direction[:5], m_wind_speed[:5], m_wind[:5]))
    while ActTime[:,-1].sum(axis=0) != 0:
        time += 1
        print('{} still burning @ time {} ########'.format(int(ActTime[:,-1].sum(axis=0)), time))
        # fire mask
        m_targetSource = mask_target_to_source(time, edgelist, ActTime)
        print("{} target to source".format(m_targetSource.sum(axis=0)))
        m_burnt = mask_burnt(time, edgelist, ActTime)
        print("{} burnt".format(m_burnt.sum(axis=0)))
        m_fire = m_burnt & m_targetSource
        print("{} fire".format(m_fire.sum(axis=0)))
        # cumulated mask
        m_propagated= m_wind & m_fire
        print("{} cumulated".format(m_propagated.sum(axis=0)))
        # append the activated layers to ActTime
        ActTime = np.c_[ActTime, m_propagated]
        # get the active edges
        ActEdges = edgelist[ActTime[:,time] == 1]
        print("ActEdges : {}".format(ActEdges.shape))

1386 ignitions : [0 0 0 0 0] over 3457222 total ~ 0.040089991328297686 % of buildings
1386 still burning @ time 1 ########
70043 target to source
1386 burnt
68 fire
0 cumulated
ActEdges : (0, 5)
1360 ignitions : [0 0 0 0 0] over 3457222 total ~ 0.03933794242892125 % of buildings
1360 still burning @ time 1 ########
67088 target to source
1360 burnt
48 fire
3 cumulated
ActEdges : (3, 5)
3 still burning @ time 2 ########
136 target to source
1360 burnt
1 fire
0 cumulated
ActEdges : (0, 5)
